In [ ]:
import pandas as pd

burns = pd.read_pickle('/home/daniele/repositories/ABM_Uni_v3/data/ethereum_v2/monthly/burns_2022-01.pkl')
burns['type'] = 'burn'
mints = pd.read_pickle('/home/daniele/repositories/ABM_Uni_v3/data/ethereum_v2/monthly/mints_2022-01.pkl')
mints['type'] = 'mint'
swaps = pd.read_pickle('/home/daniele/repositories/ABM_Uni_v3/data/ethereum_v2/monthly/swaps_2022-01.pkl')
swaps['type'] = 'swap'

df = pd.concat([burns, mints, swaps], ignore_index=True)
df = df.sort_values(by="timestamp")
df

,amount0,amount1,amountUSD,feeLiquidity,feeTo,id,liquidity,logIndex,needsComplete,sender,...,reserve0_after,reserve1_after,datetime,month,amount0In,amount0Out,amount1In,amount1Out,from,recipient
265,2500.000000,-0.662822,2492.239554,NaN,NaN,0x619aab7430c173d035685b31d6a1a129df6e1942bf4b...,NaN,175,NaN,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,...,1.040068e+08,27657.511855,2022-01-01 23:00:27,2022-01,2500.000000,0.000000,0.000000,0.662822,0xeaf8e3fb5803acb53b6e02d52b927586958d0091,0x60148519ad90ca9bb11a126f466fbfb73d895ce6
266,-9885.449411,2.636901,9885.053538,NaN,NaN,0x59d1c237e28a5321678fd52aa1445d6382abcfdc8449...,NaN,106,NaN,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,...,1.039969e+08,27660.148756,2022-01-01 23:01:09,2022-01,0.000000,9885.449411,2.636901,0.000000,0xc5f16770dcc9ce6e6f2e02f7afdc101b28926308,0xc5f16770dcc9ce6e6f2e02f7afdc101b28926308
267,-884.336591,0.235918,884.347846,NaN,NaN,0x8a1c9e40e8da9521e7f63a6b3802f928d27569a32c53...,NaN,188,NaN,0x1111111254fb6c44bac0bed2854e76f90643097d,...,1.039960e+08,27660.384674,2022-01-01 23:01:11,2022-01,0.000000,884.336591,0.235918,0.000000,0xe8ea0e623a39c5c2c8c7e9d2fd2b2049767681d9,0xa5f0cf205af1f5b02c00ba7ab834824c01855b54
268,273.982635,-0.072654,273.156339,NaN,NaN,0x049d134d76c506ba41e5746a5ca1420ad7a8ffae6fa4...,NaN,191,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,...,1.039963e+08,27660.312020,2022-01-01 23:01:34,2022-01,273.982635,0.000000,0.000000,0.072654,0xea1b50d3b19aa218a39ba746da24b2f1064a879a,0x3827f0e43bb7d52a2ddd40eb8812180394a21b0d
269,-6626.784958,1.767969,6626.920779,NaN,NaN,0xddf505a0b8bf236f159953597f6cdd08709927037478...,NaN,345,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,...,1.039897e+08,27662.079988,2022-01-01 23:01:34,2022-01,0.000000,6626.784958,1.767969,0.000000,0x2a46965a6c5fe2922abf014ebde93196e88fd2a6,0x2a46965a6c5fe2922abf014ebde93196e88fd2a6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82112,-1893.467121,0.706042,1895.253734,NaN,NaN,0xee741e3f8f196e8a2ec7d39994f643e33d1175ee7b5e...,NaN,140,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,...,9.811300e+07,36475.636415,2022-01-31 22:58:38,2022-01,0.000000,1893.467121,0.706042,0.000000,0x48ef00965a6a28848fa7040e6d39726f2eed8525,0x48ef00965a6a28848fa7040e6d39726f2eed8525
82113,-15234.808557,5.681796,15250.389916,NaN,NaN,0xa2cdd7455d261fd0fb8e5396251028bbb38fbcc067c3...,NaN,30,NaN,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,...,9.809776e+07,36481.318211,2022-01-31 22:59:20,2022-01,0.000000,15234.808557,5.681796,0.000000,0x95427a0cda907f50b3211bab32301b07a6139a86,0x220bda5c8994804ac96ebe4df184d25e5c2196d4
82114,2063.898420,-0.765219,2059.952518,NaN,NaN,0x0e835cc19366b2081d1587aebaa5805db606d257f406...,NaN,201,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,...,9.809983e+07,36480.552992,2022-01-31 22:59:39,2022-01,2063.898420,0.000000,0.000000,0.765219,0x317bc38b66566566529c41462ba774f489b4a63f,0x60a39010e4892b862d1bb6bdde908215ac5af6f3
82115,9300.412519,-3.447859,9282.140419,NaN,NaN,0xa003976b558339b7e0a6896fe174bba7a41c8c48f330...,NaN,145,NaN,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,...,9.810913e+07,36477.105133,2022-01-31 22:59:50,2022-01,9300.412519,0.000000,0.000000,3.447859,0xc1da2635153878e4d0389a51f9b6fd4635d5baff,0xdf60e6416fcf8c955fddf01148753a911f7a5905


In [1]:
import pandas as pd

df = pd.read_pickle('/home/daniele/repositories/ABM_Uni_v3/data/usdc_weth_05.pkl')
df

,eventType,blockNumber,logIndex,timestamp,transactionHash,gasUsed,sender,recipient,amount0,amount1,...,L_after,sqrt_after,tick_after,x_after,y_after,affectsActive,deltaL_applied,origin,gasPrice,effectiveGasPrice
0,Swap,16308193,144,1672531247,0x94ddda3238c96bbb12dd3b2b36e84c04ee7824fcaff9...,364517.0,0x53222470CdcfB8081C0E3a50fd106f0D69e63F20,0x53222470CdcfB8081C0E3a50fd106f0D69e63F20,-5039850865,4217481739441790046,...,39974582212968359597,2291340004214720379120476440275511,205456,1382209837991415,1156095969786673572989174,NaN,NaN,0x7a57533Bccd4C578F42580eaaA0fd6A5634473d3,1.999335e+10,1.999335e+10
1,Swap,16308193,307,1672531247,0xa7235b63f26b3e60480cfdf7a2dfda59748e8b01f2b9...,249637.0,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,23938102,-20012087556233926,...,39974582212968359597,2291339964551493463877780626424282,205456,1382209861917547,1156095949774586016755248,NaN,NaN,0xc4cC364706Fe781FB00a47BaDb297Af36A482Cc1,1.949335e+10,1.949335e+10
2,Swap,16308195,41,1672531271,0xc32c1a42d18914a139224fb015d357f952da300b322f...,261522.0,0x1111111254EEB25477B68fb85Ed929f73A960582,0x1111111254EEB25477B68fb85Ed929f73A960582,74928651,-62639830971202232,...,39974582212968359597,2291339840401635396257026212333731,205456,1382209936808733,1156095887134755045553015,NaN,NaN,0x80681fa00Ea7e84CdeB631cE96a756E33ED67d56,2.212594e+10,2.212594e+10
3,Mint,16308196,4,1672531283,0xd206569ae3c6d66f294304fd1e8391731febc20b4c2d...,514483.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,NaN,8258538328628,13297375970194539525447,...,1437495831953515783853,2291339840401635396257026212333731,205456,49704610106036316,41573493132235227728708176,True,1397521249740547424256.0,0x065e3DbaFCb2C26A978720f9eB4Bce6aD9D644a1,1.968192e+10,1.968192e+10
4,Swap,16308196,12,1672531283,0xbce4e3e514d36b4628d9d404b9914b18f06061520b74...,121338.0,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0xea7c3a066E343DA79d9381F02B7a85879999E039,-59438551150,49740000000000000000,...,1437495831953515668937,2291342580471075253351994683067231,205456,49704550667485162,41573542847365224405248427,NaN,NaN,0xea7c3a066E343DA79d9381F02B7a85879999E039,2.118192e+10,2.118192e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896865,Swap,18908889,103,1704067139,0x4d73e3e68bc1356ff8d99b22aa9441e01fde270527f1...,NaN,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x26cbCF9E87386d7E71F6314f009660dE76f22E34,-20516329,9000000000000000,...,20500328755020716289,1658984789764899465294335242686948,198997,979034520520709,429263187615088399680941,NaN,NaN,0x26cbCF9E87386d7E71F6314f009660dE76f22E34,NaN,NaN
1896866,Swap,18908891,220,1704067163,0x1bafc6e5d166d33846e09e106a7510d90cc1a8f8856c...,NaN,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,-110472284,48461430050944303,...,20500328755020716289,1658984976961424290884416183568007,198997,979034410048424,429263236052287735599771,NaN,NaN,0x92b014816b544328b1cA4D749703eAD4dd80631d,NaN,NaN
1896867,Swap,18908891,259,1704067163,0x33897c7e5a759d1438a95d057554fb43741cb9635164...,NaN,0xDef1C0ded9bec7F1a1670819833240f027b25EfF,0x5C421040e937753eCA4Ba7250fBd898B2886Eb0f,-14232128745,6243370639463194321,...,20500328755020716289,1659009093816533127246047496959153,198997,979020177919678,429269476301241879062494,NaN,NaN,0x5C421040e937753eCA4Ba7250fBd898B2886Eb0f,NaN,NaN
1896868,Swap,18908893,186,1704067187,0x60e0c0daba519ec2c35243b6db24f087c99ebe6692c1...,NaN,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x4fd39C9E151e50580779bd04B1f7eCc310079fd3,507570885,-222442435272678448,...,20500328755020716289,1659008234137372736401645585443945,198997,979020685236777,429269253858806606384045,NaN,NaN,0xD36aac0c9676e984D72823Fb662ce94D3Ab5E551,NaN,NaN


In [6]:
df = df.drop(columns='owner')
df.to_csv('usdc_weth_05.csv', index=False)

In [3]:
import pandas as pd
pd.read_parquet('/home/daniele/repositories/generative_AMM/data/ethereum/USDC_WETH_events.parquet').iloc[:1000].to_csv('/home/daniele/repositories/generative_AMM/data/ethereum/usdc_weth_05_sample.csv', index=False)

In [1]:
import pandas as pd

df = pd.read_csv('/home/daniele/repositories/ABM_Uni_v3/data/usdc_weth_05.csv')
df_gas = pd.read_csv('/home/daniele/repositories/ABM_Uni_v3/data/univ3_pool_events_with_gas.csv')

df_gas = df_gas[df_gas['transactionHash'].isin(df['transactionHash'])]
df['gasPrice'] = df_gas['gasPrice']
df['effectiveGasPrice'] = df_gas['effectiveGasPrice']

df.to_pickle('/home/daniele/repositories/ABM_Uni_v3/data/usdc_weth_05.pkl')

/tmp/ipykernel_2971196/2437493676.py:3: DtypeWarning: Columns (11,13,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/daniele/repositories/ABM_Uni_v3/data/usdc_weth_05.csv')
/tmp/ipykernel_2971196/2437493676.py:4: DtypeWarning: Columns (11,13,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gas = pd.read_csv('/home/daniele/repositories/ABM_Uni_v3/data/univ3_pool_events_with_gas.csv')


In [ ]:
import pandas as pd
df_new = pd.read_pickle('/home/daniele/repositories/ABM_Uni_v3/data/usdc_weth_05.pkl')

In [4]:
df_new[df_new['tick_before'] != df_new['tick_after']]

,eventType,blockNumber,logIndex,timestamp,transactionHash,gasUsed,sender,recipient,amount0,amount1,...,L_after,sqrt_after,tick_after,x_after,y_after,affectsActive,deltaL_applied,origin,gasPrice,effectiveGasPrice
13,Swap,16308201,14,1672531343,0xe02067b5323761fd85f25a309428368b6f80104bdc91...,216484.0,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x5063E2474f6846d31480e1F4B7697ab98522D242,-166018238333,138946141154305955615,...,39974582212968359597,2291618980688239105758670787238093,205459,1382041570914944,1156236727411516260893061,NaN,NaN,0x5063E2474f6846d31480e1F4B7697ab98522D242,1.809340e+10,1.809340e+10
43,Swap,16308248,7,1672531907,0x10a88d2b52b86f0a08eee1062a22c5c94083b6c3cb62...,244174.0,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x458e77A91D2Eb61b9dD7424de04f3eFa10E6aF44,-26736275088,22381800927376017636,...,32965484375164900522,2291781111504232339511009105018978,205460,1139635343151392,953570952866523464368256,NaN,NaN,0x458e77A91D2Eb61b9dD7424de04f3eFa10E6aF44,1.447812e+10,1.447812e+10
48,Swap,16308257,115,1672532015,0xea5348d42b1c46c5c1fbe4d0fbf368722ed1205b02f5...,142040.0,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x2eA32F87107e50A168ea0834bc92501226E1279F,-1500000000,1255800550790446314,...,32965484375164900522,2291848859659647636672557768153462,205461,1139601654982114,953599141716143536355333,NaN,NaN,0x2eA32F87107e50A168ea0834bc92501226E1279F,1.394635e+10,1.394635e+10
52,Swap,16308264,5,1672532099,0x02b53bf52b24241cddec003d8160cb2873829eef2a9b...,121822.0,0x0087BB802D9C0e343F00510000729031ce00bf27,0x0087BB802D9C0e343F00510000729031ce00bf27,-173327977956,145134436468699987968,...,32965484375164900522,2292212692375916116211815858983537,205464,1139420771084658,953750526291308496997638,NaN,NaN,0xBcb3f1481D599A92B1c8D5035d3ac0E88b097bf8,2.723282e+11,2.723282e+11
54,Swap,16308265,7,1672532111,0xd729a2fc102be3ec594e501f9c0219c45964f17db81e...,120836.0,0x0087BB802D9C0e343F00510000729031ce00bf27,0x0087BB802D9C0e343F00510000729031ce00bf27,-67559290532,56582577434099998720,...,32965484375164900522,2292359261166749796450909947845653,205465,1139347918836976,953811511060243969798532,NaN,NaN,0x1cAD683486c7c6f3A21D9A522e5D5CBb02D9C35E,5.253066e+10,5.253066e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896847,Swap,18908864,16,1704066839,0x556608a3b9ff634394f4e7a6a424da0d67c7c7103cbc...,NaN,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,135112687877,-59190394275042864565,...,20500328755020716289,1658577565692199833204077713531315,198992,979274899043249,429157818171921042504381,NaN,NaN,0xcCDD43c18B417F3EF046aFc4bc773B70907bA2D3,1.142796e+10,1.142796e+10
1896849,Swap,18908865,24,1704066851,0xd6d11ea5b7e551b3ae7b3d1f796f23a363cdf641f50c...,NaN,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,-119184517412,52263746238553576783,...,20500328755020716289,1658776064123575552291258333211650,198995,979157713525837,429209179745525952520278,NaN,NaN,0x042523DB4F3Effc33d2742022B2490258494f8B3,1.224282e+10,1.224282e+10
1896856,Swap,18908875,21,1704066971,0x91b6359a15017baccf639a931542c4b8663cf6ec23bd...,NaN,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,-105434049584,46247822530074747340,...,20500328755020716289,1659012156059369037191533007491570,198997,979018370821692,429270268658187587078799,NaN,NaN,0x76e40D0a69Fd81826B5eb7D18145626d46EAFdef,1.186703e+10,1.186703e+10
1896860,Swap,18908881,225,1704067043,0xafbc9e5980970de7293ce821b4297d88b97f18cf1910...,NaN,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x51F3c2a348CEead4fDd46Fa59Dc64fC7C88cc0da,-1139755784,500000000000000000,...,20500328755020716289,1659015086763198592129035413208566,198998,979016641354063,429271026979346160679044,NaN,NaN,0x7f8DA5FBD700a134842109c54ABA576D5c3712b8,NaN,NaN
